Import relevant packages:

In [1]:
import spacy
import re
import pandas as pd

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/Lassenoerre/datascience/master/datasets/articles_0_6000.csv?token=ARZVGJ34RODPQRSDIMC6LAC7XE3II', header=0, names=['Title', 'Topic', 'Date', 'Content', 'URL'], usecols=[1,2,3,4,5])

NameError: ignored

In [7]:
df.head()

,Title,Topic,Date,Content,URL
0,Shopping Malls Tracking Holiday Sales,Street Signs,27-11-2006,Its not only retailers who are closely watchin...,https://www.cnbc.com/2006/11/27/shopping-malls...
1,Consumers Adore Gift Cards,U.S. News,27-11-2006,The National Retail Federation\xe2\x80\x99s Ho...,https://www.cnbc.com/2006/11/27/consumers-ador...
2,What's Wrong With Wal-Mart?,Street Signs,27-11-2006,Despite the fine weather in much of the nation...,https://www.cnbc.com/2006/11/27/whats-wrong-wi...
3,"Shape Up, Ship Out",Holiday Central,27-11-2006,It helped nail the coffin shut on eToys. It\xe...,https://www.cnbc.com/2006/11/27/shape-up-ship-...
4,Big Box Boom: Gadgets Sell \xe2\x80\x93 And Pr...,Holiday Central,27-11-2006,There\xe2\x80\x99s plenty of time to shop for ...,https://www.cnbc.com/2006/11/27/big-box-boom-g...


Convert all article content to stringe type:

In [8]:
df['Content'] = df['Content'].astype('str')

Replace all article content with the title if the title is longer than the article content: 

In [71]:
for index in df.index:
  if len(df['Content'][index]) < len(df['Title'][index]):
    df['Content'][index]=df['Title'][index]

Define a function to remove special unicode characters and other non-relevant N-grams from each article:

In [55]:
def remove_clutter(text):

    #Trying to remove special unicode characters
    #text = re.sub(r'\\x[A-Za-z0-9_]{2}', '', text)
    
    #Trying to remove video annotation
    text = re.sub(r'VIDEO([0-9]|[0-9]{2}):[0-9]{4}:[0-9]{2}', ' ', text)

    # Remove unicode characters:
    text = ''.join([x for x in text if ord(x) < 127])
    
    # Remove all "Getty images":
    text = re.sub(r'getty images','', text) 

    # Remove commas:
    text = re.sub(r',','', text)

    # Replace abbrevations:
    text = re.sub(r"don't","dont", text)
    text = re.sub(r"can't","cant", text)
    text = re.sub(r"weren't","werent", text)
    text = re.sub(r"couldn't","couldnt", text)
    text = re.sub(r"doesn't","doesnt", text)
    text = re.sub(r"didn't","didnt", text)
    
     # Remove all double-spaces:
    text = re.sub(r'  ',' ', text)

    return text

In [57]:
remove_clutter("didn't can't weren't getty imgages  â Â ")

'didnt cant werent getty imgages  '

Load the English language model instance in spaCy:

In [3]:
nlp = spacy.load('en_core_web_sm')

Define a function to clean all our articles in a specific column in a dataset and return the tokenized data


In [4]:
def cleaning(df,column):
    
    # Create an empty list to store tokens:
    tokens = []

    # Apply the remove_clutter function on all articles in the df:
    df[column].apply(remove_clutter)

    # Define an object to count the progress:

    count = 0
    
    # Iterate over all articles in the dataframe and create a nlp object for each:
    for article in nlp.pipe(df[column], disable=["parser"]):
        
        # Store all cleaned tokens in a list using list comprehension:
        article_tok = [token.lemma_.lower() for token in article if token.is_alpha and not token.is_stop and token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV', 'VERB'] and token.ent_type_ not in ['PERSON', 'MONEY', 'PERCENT', 'LOC', 'DATE', 'TIME', 'QUANTITY', 'ORDINAL'] and len(token)>1]
        
        # Insert list of tokens into the list "tokens":
        tokens.append(article_tok)

        # Print the progress:
        count += 1
        print(f'processed {count}/{len(df[column])}')
        
    # Insert a column in the dataframe with all tokens for each article:
    df['tokens'] = tokens
    
    # Insert tokens as a string in a new column in the dataframe:
    df["clean_articles"] = df["tokens"].map(lambda row: " ".join(row))

    # Return the df
    return df

In [17]:
test = "this is a test, Frederik, Wednesday, 100m USD, tomorrow, najarian"
article = nlp(test)

def test:
    doc = nlp(text)
    for token in doc:
        if token.is_alpha and not token.is_stop and token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV', 'VERB'] and token.ent_type_ not in ['PERSON', 'MONEY', 'PERCENT', 'LOC', 'DATE', 'TIME', 'QUANTITY', 'ORDINAL'] and len(token)>1:
            print(toke.text)

    
    article_tok = [token.lemma_.lower() for token in article if token.is_alpha and not token.is_stop and token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV', 'VERB'] and token.ent_type_ not in ['PERSON', 'MONEY', 'PERCENT', 'LOC', 'DATE', 'TIME', 'QUANTITY', 'ORDINAL'] and len(token)>1]

article_string =  ""

for token in article_tok:
    article_string.join()
article_tok.map(lambda row: " ".join(row))

for token in article_tok:
    nlp(token)
    print(token.text, token.ent_type_)


#[tok.lemma_ for tok in nlp(article_tok)]

AttributeError: 'list' object has no attribute 'map'

In [60]:
def test(text):
    doc = nlp(text)
    for i, token in enumerate(doc, start=1):
        if token.is_alpha and not token.is_stop and token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV', 'VERB'] and token.ent_type_ not in ['PERSON', 'MONEY', 'PERCENT', 'LOC', 'DATE', 'TIME', 'QUANTITY', 'ORDINAL'] and len(token)>1:
            print(i, token.text, token.pos_, token.ent_type_)
        else:
            print(i, f"rejected {token.text}, {token.pos_}, {token.ent_type_}")

test("this is a test, Frederik, Wednesday, 100m USD, tomorrow, najarian, ijr finerman, dis, xhb finerman, Frederik Permin")
    

1 rejected this, DET, 
2 rejected is, AUX, 
3 rejected a, DET, 
4 test NOUN 
5 rejected ,, PUNCT, 
6 rejected Frederik, PROPN, PERSON
7 rejected ,, PUNCT, 
8 rejected Wednesday, PROPN, DATE
9 rejected ,, PUNCT, 
10 rejected 100, NUM, CARDINAL
11 rejected m, PROPN, 
12 USD NOUN 
13 rejected ,, PUNCT, 
14 rejected tomorrow, NOUN, DATE
15 rejected ,, PUNCT, 
16 najarian PROPN ORG
17 rejected ,, PUNCT, 
18 rejected ijr, PROPN, PERSON
19 rejected finerman, PROPN, PERSON
20 rejected ,, PUNCT, 
21 rejected dis, PROPN, PERSON
22 rejected ,, PUNCT, 
23 rejected xhb, PROPN, PERSON
24 rejected finerman, PROPN, PERSON
25 rejected ,, PUNCT, 
26 rejected Frederik, PROPN, PERSON
27 rejected Permin, PROPN, PERSON


Call the cleaning function on the dataset:

In [3]:
cleaning(df,'Content')


NameError: ignored

Example: How to entity recognition work:

In [50]:
df['URL'][5167]

'https://www.cnbc.com/2007/06/06/prudential-shuts-down-stock-research-trading-arm.html'

In [57]:
doc = nlp("don't")

In [58]:
for tok in doc:
  print(tok)

do
n't


In [ ]:
sentence_spans = list(doc.sents)
displacy.serve(sentence_spans, style="dep")

displacy.serve(doc, style="dep")